In [1]:
import cv2
import os
from os import listdir
import numpy as np
from stitching import Stitcher

In [2]:
grails = []
folder_dir = "lounge"
for image in sorted(os.listdir(folder_dir)):
    if (image.endswith(".jpg")):
        img = cv2.imread(folder_dir+"/"+image)
        grails.append(img)
stitcher = Stitcher(
    detector="sift",
    confidence_threshold=0.01,  
    matcher_type='homography',
    range_width=-1,
    crop=True
)

try:
    # Add error handling for the stitching process
    panorama = stitcher.stitch(grails)
    if panorama is not None:
        cv2.imwrite(folder_dir+'_pano.jpg', panorama)
    else:
        print("Stitching failed - no panorama was created")
except Exception as e:
    print(f"Error during stitching: {str(e)}")

In [3]:
def resize(path, new_path):
    image = cv2.imread(path)
    height, width, dim = image.shape
    bot_mid = np.mean(image[-1:, :, :], axis=(0, 1))
    top_mid = np.mean(image[:1, :, :], axis=(0, 1))
    new_height = width//2 #equirectangular image is 2:1
    new_image = bot_mid*np.ones((new_height, width, dim), dtype=np.uint8) #creating blank image
    new_image[:new_height//2, :, :] = top_mid*np.ones((new_height//2, width, dim), dtype=np.uint8)
    bot, top = (new_height-height)//2, (new_height+height)//2 #center original image in center
    new_image[bot:top, : , :] = image
    cv2.imwrite(new_path, new_image)

resize(folder_dir+"_pano.jpg", folder_dir+"_resized.jpg")